# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Defining the problem ----

Building a model that predicts salaries based on job descriptions

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
print('Libraries loaded')

#your info here
__author__ = "Kevin Moldovan"
__email__ = "email@gmail.com"

## Part 2 - DISCOVER

### ---- 2 Loading the data ----

In [3]:
#load the data into a Pandas dataframe
train_features = pd.read_csv('data/train_features.csv')
train_salaries = pd.read_csv('data/train_salaries.csv')
test_features= pd.read_csv('data/test_features.csv')
print('CSVs loaded')

CSVs loaded


In [ ]:
#only for goolge colab - mounting google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#only for google colab - loading data
train_features = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/train_features.csv')
train_salaries = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/train_salaries.csv')
test_features = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/test_features.csv')
print('CSVs loaded')

### ---- 3 Clean the data ---- 
Looking for duplicate data, invalid data (e.g. salaries <=0), or corrupt data and remove it

#### Checking for duplicate data

In [4]:
Df1dup = train_features.duplicated().sum()
Df2dup = train_salaries.duplicated().sum()
Df3dup = test_features.duplicated().sum()
print('train_features duplicates: ', Df1dup)
print('train_salaries duplicates: ', Df2dup)
print('test_features  duplicates: ', Df3dup)

train_features duplicates:  0
train_salaries duplicates:  0
test_features  duplicates:  0


#### Finding Corrupt Data

In [7]:
train_features.head(10)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16
5,JOB1362684407692,COMP15,MANAGER,DOCTORAL,COMPSCI,FINANCE,2,31
6,JOB1362684407693,COMP15,CFO,NONE,NONE,HEALTH,23,24
7,JOB1362684407694,COMP24,JUNIOR,BACHELORS,CHEMISTRY,EDUCATION,9,70
8,JOB1362684407695,COMP20,JANITOR,HIGH_SCHOOL,NONE,EDUCATION,1,54
9,JOB1362684407696,COMP41,VICE_PRESIDENT,BACHELORS,CHEMISTRY,AUTO,17,68


In [8]:
test_features.head(10)

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44
5,JOB1362685407692,COMP40,CTO,MASTERS,COMPSCI,FINANCE,6,23
6,JOB1362685407693,COMP32,SENIOR,MASTERS,COMPSCI,SERVICE,6,32
7,JOB1362685407694,COMP11,CEO,BACHELORS,BIOLOGY,SERVICE,9,73
8,JOB1362685407695,COMP39,MANAGER,DOCTORAL,PHYSICS,HEALTH,6,78
9,JOB1362685407696,COMP22,VICE_PRESIDENT,NONE,NONE,AUTO,23,97


In [9]:
train_salaries.head(10)

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163
5,JOB1362684407692,113
6,JOB1362684407693,178
7,JOB1362684407694,73
8,JOB1362684407695,31
9,JOB1362684407696,104


In [11]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [14]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [16]:
train_salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   jobId   1000000 non-null  object
 1   salary  1000000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.3+ MB


In [12]:
train_features.columns

Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')

In [12]:
#Merging Train and Target DFs
train_df = pd.merge(train_features, train_salaries, on='jobId')

In [13]:
#Dropping zero salaries
train_df.drop(train_df[train_df.salary < 8.5].index, inplace = True)

In [14]:
train_df.shape

(999995, 9)

In [8]:
#Find invalid data
#dropsal = train_salaries[train_salaries.salary < 8.5]
#train_salaries = train_salaries.drop(    )
#df.drop(df[df['Age'] < 25].index, inplace = True)

### ---- 4 Explore the data (EDA) ----

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

In [16]:
train_df

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163
...,...,...,...,...,...,...,...,...,...
999995,JOB1362685407682,COMP56,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,19,94,88
999996,JOB1362685407683,COMP24,CTO,HIGH_SCHOOL,NONE,FINANCE,12,35,160
999997,JOB1362685407684,COMP23,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,16,81,64
999998,JOB1362685407685,COMP3,CFO,MASTERS,NONE,HEALTH,6,5,149


### ---- 5 Establish a baseline ----

In [5]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

In [6]:
df_baseline = train_df.groupby('jobType', as_index = False).mean()
df_baseline.rename(columns = {'salary':'avg_salary'}, inplace = True)
df_baseline = df_baseline[['jobType', 'avg_salary']]
df_baseline

,jobType,avg_salary
0,CEO,145.311425
1,CFO,135.458547
2,CTO,135.479983
3,JANITOR,70.813045
4,JUNIOR,95.331557
5,MANAGER,115.367596
6,SENIOR,105.487775
7,VICE_PRESIDENT,125.367629


### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

In [ ]:
#getting dummy values for categorical variables 

In [ ]:
#scaling min max values 
#when do you use min max?

In [15]:
#defining train and test 
x = train_df.iloc[:,:-1]
x

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16
...,...,...,...,...,...,...,...,...
999995,JOB1362685407682,COMP56,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,19,94
999996,JOB1362685407683,COMP24,CTO,HIGH_SCHOOL,NONE,FINANCE,12,35
999997,JOB1362685407684,COMP23,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,16,81
999998,JOB1362685407685,COMP3,CFO,MASTERS,NONE,HEALTH,6,5


In [16]:
#dropping jobid and company id because no correlation

x = x.drop(columns=['jobId','companyId'])
x

,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,CFO,MASTERS,MATH,HEALTH,10,83
1,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16
...,...,...,...,...,...,...
999995,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,19,94
999996,CTO,HIGH_SCHOOL,NONE,FINANCE,12,35
999997,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,16,81
999998,CFO,MASTERS,NONE,HEALTH,6,5


In [17]:
#getting dummy variables for categorical columns 
x = pd.get_dummies(data=x, columns=['jobType' , 'degree' , 'major', 'industry'])

In [18]:
x

,yearsExperience,milesFromMetropolis,jobType_CEO,jobType_CFO,jobType_CTO,jobType_JANITOR,jobType_JUNIOR,jobType_MANAGER,jobType_SENIOR,jobType_VICE_PRESIDENT,degree_BACHELORS,degree_DOCTORAL,degree_HIGH_SCHOOL,degree_MASTERS,degree_NONE,major_BIOLOGY,major_BUSINESS,major_CHEMISTRY,major_COMPSCI,major_ENGINEERING,major_LITERATURE,major_MATH,major_NONE,major_PHYSICS,industry_AUTO,industry_EDUCATION,industry_FINANCE,industry_HEALTH,industry_OIL,industry_SERVICE,industry_WEB
0,10,83,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,3,73,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,10,38,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,8,17,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,8,16,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,19,94,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
999996,12,35,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
999997,16,81,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
999998,6,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0


In [19]:
scaler = MinMaxScaler()
x[['yearsExperience', 'milesFromMetropolis']] = scaler.fit_transform(x[['yearsExperience', 'milesFromMetropolis']])

In [20]:
y = train_df.iloc[:,-1:].values

### ---- 8 Create models ----

In [15]:
#create and tune the models that you brainstormed during part 2

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [22]:
print('starting...')
# Create linear regression object
lr = linear_model.LinearRegression()

# Train the model using the training sets
lr.fit(x_train, y_train)

# Make predictions using the testing set
y_pred = lr.predict(x_test)

# Getting Mean Squared Error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 384.80


In [25]:
print('starting...')
# create regressor object
regressor = RandomForestRegressor(n_estimators = 100, max_depth = 15, max_features = 15, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(x_train, y_train.ravel())  

# Make predictions using the testing set
y_pred = regressor.predict(x_test)  

# Getting Mean Squared Error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 376.19


In [ ]:
from sklearn.cross_validation import cross_val_score 
scores = cross_val_score(estimator=pipe_lr, X=X_train, y=Y_train, cv=12, n_jobs=)
mean_scores = scores.mean()

In [ ]:
from sklearn.learning_curve import validation_curve
number_of_trees= [1,2,3,4,5,6,7,99,1000]
train_scores, test_scores = validation_curve(estimator=, … X=X_train,y=Y_train, param_range=number_of_trees, …)

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
# getting best hyperparameters

In [ ]:
# tuning model 

In [12]:
reg = GradientBoostingRegressor(learning_rate = 1, max_depth = 5, max_features = 30, min_samples_split = 10, n_estimators = 150, random_state = 0)
reg.fit(x_train, y_train.ravel())
y_pred = reg.predict(x_test)
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 374.22


In [ ]:
param = {'max_depth': [5, 10, 20, None],
         'learning_rate':[.01, 1.0, 10], 
         'n_estimators':[50, 100, 150], 
         'max_features':[10, 20, 30],
         'min_samples_split': [5, 10, 20]}

In [ ]:
rnd_search = RandomizedSearchCV(GradientBoostingRegressor(), param, 
n_iter =10, cv=2)
rnd_search.fit(x,y.ravel())
rnd_search.best_params_
rnd_search.best_score_

In [ ]:
rnd_search.best_params_

In [ ]:
rnd_search.best_score_

In [ ]:
clr=gridsearchcv

### ---- 9 Test models ----

In [1]:
#do 5-fold cross validation on models and measure MSE

In [14]:
cvlr = cross_val_score(lr, x, y, cv=5, scoring='neg_mean_squared_error')
mselr = np.average(cvlr)
print(mselr)

-384.49516949432666


In [24]:
cross_val_score(regressor, x, y.ravel(), cv=5, scoring='neg_mean_squared_error')
mserf = 

array([-453.89056486, -453.6485728 , -455.30029597, -451.57639736,
       -452.19440379])

In [ ]:
cvgb = cross_val_score(reg, x, y.ravel(), cv=5, scoring='neg_mean_squared_error')
msegb = np.average(cvgb) 
print(msegb)

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "production" model
#how is everyone else doing this 

In [24]:
k = 800
print(k)

800


In [27]:
list9 = [mselr, k]
list9

[-384.49516949432666, 800]

In [17]:
type('mselr')

str

In [29]:
MSEdf = pd.DataFrame(columns = ['Model', 'MSE'])
MSEdf

,Model,MSE


In [37]:
MSEdf.loc[len(MSEdf.index)] = ['LR', mselr] 
MSEdf

,Model,MSE
0,LR,-384.495169


In [38]:
MSEdf.loc[len(MSEdf.index)] = ['reg', msegb]
MSEdf

NameError: name 'mserf' is not defined

In [ ]:
MSEdf.loc[len(MSEdf.index)] = ['Regressor', mserf]
MSEdf

In [39]:
MSEdf.min()

Model            LR
MSE     -384.495169
dtype: object

In [ ]:
bestmodel = min(list9)
print("The best model is: ", bestmodel)

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [16]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

In [ ]:
#prepare test features
test_features.info()

In [ ]:
test_features.head()

In [ ]:
#dropping unneeded columns
test_features = test_features.drop(columns=['jobId','companyId'])
test_features.head()

In [ ]:
#converting categorical columns
test_features = pd.get_dummies(data=test_features, columns=['jobType' , 'degree' , 'major', 'industry'])
test_features.head()

In [ ]:
#scaling numerical columns 
test_features[['yearsExperience', 'milesFromMetropolis']] = scaler.fit_transform(test_features[['yearsExperience', 'milesFromMetropolis']])

In [ ]:
test_features.head()

In [ ]:
#prediction = reg.predict(test_feature)
#prediction = pd.DataFrame(predictions, columns=['predictions']).to_csv('prediction.csv')

In [ ]:
#predicting based on test set
#y_pred = reg.predict(test_feature)
#print('Mean squared error: %.2f'
#      % mean_squared_error(y_test, y_pred))

In [ ]:
#predicting based on test set
y_predtest = reg.predict(test_features)

In [ ]:
#saving prediction to csv
y_predtest = pd.DataFrame(y_predtest, columns=['predictions']).to_csv('prediction.csv')

In [ ]:
predictions = pd.read_csv('prediction.csv')
predictions

In [ ]:
#visualization of predictions 
plt.plot(predictions.predictions)
#sns.boxplot(x = 'yearsExperience', y = 'salary', data = train_df)
plt.show()

In [ ]:
#visualizing feature importances

# get importance
importance = reg.feature_importances_
importance
# summarize feature importance
#for i,v in enumerate(importance):
#	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
#plt.bar([x for x in range(len(importance))], importance)
#plt.show()

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data